In [1]:
import pandas as pd

FILE = '../../other_sources/downloaded_full_set_metadata.zip'
SEPARATOR = ',' # '\t' for TSV
COMPOSER_MODE = True

In [5]:
dtype = 'string' if COMPOSER_MODE else dict(original=bool, terminated='boolean', converted='Int64', features='Int64', last_error='string')
df = pd.read_csv(FILE,
                 sep=SEPARATOR,
                 escapechar='\\',
                 index_col=0,
                 dtype=dtype)
N = len(df.index)
print(f"N = {N}")
df.head()

N = 1504109


,title,description,composer
ID,,,
4939836,"Springlek efter Perbj\""ors Erik Persson",Note Length 1/8 Rhythm polska K1 polska K1 Key...,<NA>
5401399,Excerpts in C,<NA>,<NA>
5925434,Wii Shoppin' for the Big Bands; Wii Shop Chann...,This is meant to be printed for use in Jazz Bi...,Composed by Kazumi Totaka; Arranged by Lars Ka...
3587506,Hide And Seek,"FROM MY OWN hide and seek ""movie not released""",Brandon
5161684,The First Love Dream,Note Length - 1/8 Key - G Meter - 4/4,<NA>


In [6]:
if COMPOSER_MODE:
  
  display(df.notna().sum() / N)

title          0.999993
description    0.496666
composer       0.580897
dtype: float64

In [7]:
if not COMPOSER_MODE:
  missing_originals = (~df.original).sum()
  n_converted = df.converted.notna().sum()
  n_features = df.features.notna().sum()
  print(f"missing original scores (based on available JSON files): {missing_originals}\n"
        f"successfully converted: {n_converted}/{N} ({n_converted/N:.2%})\n"
        f"missing files + converted files + conversion errors = {missing_originals + n_converted + 105099}\n"
        f"extracted features: {n_features}/{N} ({n_features/N:.2%})\n")

# Threshold
No unknowns, one composer per line

In [1]:
from ray_NER import basic_clean
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('../../other_sources/downloaded_full_set_metadata.zip',
                 sep=',',
                 escapechar='\\',
                 index_col=0,)

In [3]:
data_comp=data['composer'].apply(lambda x: basic_clean(str(x)))
data_comp

ID
4939836             None
5401399             None
5925434    K. T. L. Kahn
3587506          Brandon
5161684             None
               ...      
152387              None
4907851             None
5334329        Ballinger
4694386             None
6414534      K. Konvičný
Name: composer, Length: 1504109, dtype: object

In [4]:
list_comp = data_comp[~data_comp.isnull()].values
uniq, counts = np.unique(list_comp, return_counts=True)
sorted_idx = counts.argsort()
sorted_uniq= uniq[sorted_idx[::-1]]
sorted_counts=counts[sorted_idx[::-1]]

In [5]:
threshold = 10
top = sorted_uniq[:threshold]
sorted_uniq[:threshold], sorted_counts[:threshold]

(array(['J. S. Bach', 'W. A. Mozart', 'L. V. Beethoven', 'P. F. Carera',
        'Y. K. Kondo', 'T. Fox', 'J. Williams', 'Y. K.  加. 洋平',
        'Y. K.  加藤洋平', 'Urheber'], dtype=object),
 array([6844, 5238, 4577, 4344, 4292, 3185, 2811, 2801, 2353, 1494]))

In [12]:
top_IDs=data_comp[data_comp.apply(lambda x: x in top)==True]

In [ ]:
len(top_IDs)

1900

In [ ]:
top_IDs.to_csv("./best_10_composers_ID.tsv", sep='\t', index=False)
